# Prepare Power Flow Data

This notebook presents the process of data preparation for building power flow regimes. The process is separated into stages (see, [DVC config](../dvc.yaml)):
 - "parse" --- extract necessary parameters from raw data
 - "prepare" --- combine and convert data to build final dataset
 - "model" --- build power flow cases.

The naming of variables in the final dataset corresponds to the [project convention](../convention.md).

Since the NREL-118 dataset is mostly intended for OPF tasks, it skips a lot of information that is not necessary to solve these kind of tasks. Thus, to append the data with missing info, [JEAS-118 dataset](http://motor.ece.iit.edu/data/JEAS_IEEE118.doc) is used as the primary source of NREL-118. The following sections describe all the stages of data processing and all the decisions made when preparing the final dataset.

In [1]:
import os

from src.data import (
    parse_nrel118_buses,
    prepare_buses,
    parse_nrel118_lines,
    parse_jeas118_lines,
    parse_jeas118_trafos,
    prepare_branches,
)

## Buses

To build a power flow model, the following information about buses is necessary:
- rated voltage level
- in service or out of service
- name (optional)
- region (optional)

In [2]:
# Load and parse bus data of NREL-118 power system
path_nrel118_buses = os.path.join(
    "..", "data", "raw", "nrel118", "additional-files-mti-118", "Buses.csv"
)
buses_nrel118 = parse_nrel118_buses(path_nrel118_buses)
buses_nrel118.head(2)

,name,region,load_participation_factor
0,bus_1,r1,0.047169
1,bus_2,r1,0.018496


The NREL-118 dataset contains only names and regions of buses ("load_participation_factor" is for load modelling). To add missing values, it is assumed the following:
- all buses are in service
- rated voltage level of buses 8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81, 116 equals to 345 kV, the rest of buses has the voltage level of 138 kV. This corresponds to transformers data from JEAS-118 dataset.

Thus, the final bus data look as follows:

In [3]:
buses = prepare_buses(buses_nrel118)
buses.head(2)

,name,region,in_service,v_rated__kv
0,bus_1,r1,True,138
1,bus_2,r1,True,138


## Branches


Branches is a common term both for lines and transformers. The following parameters about branches are necessary to build models:
- start / end buses of the branch
- number of parallel branch systems
- resistance
- reactance
- active conductance
- in service or out of service
- maximum power flow (optional)
- transformation ratio if the branch is a transformer (optional)
- name (optional)

In [4]:
# Load and parse line data of NREL-118 power system
path_nrel118_lines = os.path.join(
    "..", "data", "raw", "nrel118", "additional-files-mti-118", "Lines.csv"
)
lines_nrel118 = parse_nrel118_lines(path_nrel118_lines)
lines_nrel118.head(2)

,name,from_bus,to_bus,max_p__mw,x__pu,r__pu
0,branch_1,bus_1,bus_2,600,0.0999,0.0303
1,branch_2,bus_1,bus_3,600,0.0424,0.0129


Since the information about active conductance and parallel number is skipped in the NREL-118 dataset, let's load it from the JEAS-118 dataset.

In [5]:
path_jeas118_lines = os.path.join("..", "data", "raw", "jeas118", "JEAS_IEEE118.doc")
lines_jeas118 = parse_jeas118_lines(path_jeas118_lines)
lines_jeas118.head(2)

,name,from_bus,to_bus,parallel,b__pu
0,branch_1,bus_1,bus_2,1,0.0254
1,branch_2,bus_1,bus_3,1,0.01082


In the NREL-118 dataset, transformers are presented as lines without values of transformation ratio. Therefore, these values will be loaded from JEAS-118 dataset.

In [6]:
path_jeas118_trafos = os.path.join("..", "data", "raw", "jeas118", "JEAS_IEEE118.doc")
trafos_jeas118 = parse_jeas118_trafos(path_jeas118_trafos)
trafos_jeas118.head(2)

,name,from_bus,to_bus,parallel,trafo_ratio
0,trafo_1,bus_8,bus_5,1,0.985
1,trafo_2,bus_26,bus_25,1,0.96


Thus, the final branch data look as follows:

In [7]:
branches = prepare_branches(lines_nrel118, lines_jeas118, trafos_jeas118, buses)
branches.head(2)

,name,from_bus,to_bus,parallel,in_service,r__ohm,x__ohm,b__µs,trafo_ratio,max_i__ka
0,branch_1,bus_1,bus_2,1,True,5.770332,19.024956,133.375341,NaN,2.510219
1,branch_2,bus_1,bus_3,1,True,2.456676,8.074656,56.815795,NaN,2.510219
